In [ ]:
import logging
import gensim
from gensim import corpora
import sys
from math import log

reload(sys)  
sys.setdefaultencoding('utf-8')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def initiate():
	Documents=[]
	Stoplist=[]
	#initiate doc
	f=open('./phc.txt')
	for line in f.readlines():
		line=line.strip()
		line=line.strip("'")
		Documents.append(line)
	#initiate stop lists
	f2=open('./stopwords.txt')
	for line in f2.readlines():
		line=line.strip('\n')
		Stoplist.append(line)
	return Documents,Stoplist

def BagOfWord(Documents,Stoplist):
	#tokenize
	Texts = [[word for word in document.lower().split() if word not in Stoplist] for document in Documents]
	#initiate keyword lists
	Frequency={}
	for line in Texts:
		n=0
		for word in line:
			Frequency[word]=Frequency.get(word,0)+1
	#delete words that only appear once
	for key in Frequency.keys():
		if Frequency[key]==1 or key[0:4]=='http':
			del Frequency[key]
	#update Texts
	Texts=[[word for word in document if word in Frequency.keys()] for document in Texts]
	#build corpus
	dictionary = corpora.Dictionary(Texts)
	dictionary.save('./AMR-words.dict')
	corpus = [dictionary.doc2bow(text) for text in Texts]
	corpora.MmCorpus.serialize('./AMR-corpus.mm', corpus)
	print(dictionary.token2id)
	return dictionary,corpus

def output(corpus,dictionary,lda):
	global topicnum
	#print doc-topic
	f3=open('./AMR-doc-topic.txt','w')
	n=0
	for line in corpus:
		topiclist=lda.get_document_topics(line)
		corpus[n].append(topiclist[0][0])
		f3.write('%d,%d,%f\n'%(n,topiclist[0][0],topiclist[0][1]))
		n=n+1
	f3.close
	#print topic-words
	f4=open('./AMR-topic-words.txt','w')
	word2id=dictionary.token2id
	word2id= sorted(word2id.iteritems(), key=lambda d:d[1], reverse = False) #from small to large
	words=[i[0] for i in word2id]
	for i in range(topicnum):
		wordlist=lda.get_topic_terms(i,topn=20)
		liststring=''
		for n in range(20):
			Id=wordlist[n][0]
			word=words[Id]
			liststring=liststring+','+word#+','+unicode(wordlist[n][1])
		f4.write('%d%s\n'%(i,liststring))
	f4.close
	return corpus,word2id

def SubMat(corpus,wordID):
	Mat=[] #2 tuple (appear or not,topic)
	n=1
	for document in corpus:
		for word in document[:-1]:
			if wordID in word:
				Mat.append((1,document[-1]))
				break
		if n==len(Mat):
			Mat.append((0,document[-1]))
		n=n+1
	return Mat

def calEntropy(Mat,topic): #bigger is better
	MI=0
	for i in range(0,2): #word is present or absent
		sublist=[a[1] for a in Mat if a[0] == i] #topic class of docs with/without word appers in
		n=sublist.count(topic)
		m=len(Mat)-n
		k=len(sublist)-n
		p1=float(n+1)/(len(Mat)+2) #P(X1,Y) --X1:belong to topic X, X0 not belong,Y:word shows or not
		p2=float(n+1)/(len(sublist)+2) #P(X1|Y)
		p3=float(m+1)/(len(Mat)+2) #P(X0,Y)
		p4=float(k+1)/(len(sublist)+2) #P(X0|Y)
		MI=MI+p1*log(p2,2)+p3*log(p4,2)
	return MI

def filterEntropy(Mat,topic): #bigger is better
	MI=0
	for i in range(0,2): #word is present or absent
		sublist=[a[1] for a in Mat if a[0] == i] #topic class of docs with/without word appers in
		n=sublist.count(topic)
		m=len(Mat)-n
		k=len(sublist)-n
		p1=float(n+1)/(len(Mat)+2) #P(X1,Y) --X1:belong to topic X, X0 not belong,Y:word shows or not
		p2=float(n+1)/(len(sublist)+2) #P(X1|Y)
		p3=float(m+1)/(len(Mat)+2) #P(X0,Y)
		p4=float(k+1)/(len(sublist)+2) #P(X0|Y)
		MI=MI+p1*log(p2,2)+p3*log(p4,2)
	if p2>p4:
		return 1
	else:
		return 0

Documents,Stoplist=initiate()
BagOfWord(Documents,Stoplist)
dictionary,corpus=BagOfWord(Documents,Stoplist)
topicnum=5
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=topicnum, update_every=1, chunksize=10000, passes=1)
corpus,word2id=output(corpus,dictionary,lda)
MI_Matrix=[[0]*topicnum]*len(word2id)
MI_Matrix2=[[0]*topicnum]*len(word2id)
for n in range(len(word2id)):
	Id=word2id[n][1]
	Mat=SubMat(corpus,Id)
	for i in range(topicnum): #iterate topics
		MI_Matrix[n][i]=calEntropy(Mat,i)
		MI_Matrix2[n][i]=filterEntropy(Mat,i)
	print 'MI',word2id[n][0],MI_Matrix[n]
	print 'filter',word2id[n][0],MI_Matrix2[n]
